In [1]:
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset, random_split 
from torch.nn import functional as F  
import torch.optim as optim
from torchsummary import summary 

 
import numpy as np
import matplotlib
import matplotlib.pyplot as mplot
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap
import os
import subprocess
import sklearn
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.datasets import load_files
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.model_selection import train_test_split, KFold
from sklearn.metrics import classification_report, confusion_matrix, ConfusionMatrixDisplay, accuracy_score, f1_score, precision_score, recall_score,  precision_recall_fscore_support
from sklearn.metrics import roc_auc_score
from sklearn.metrics import roc_curve
 
 # Convert to PyTorch tensors
import torch
import torch.nn as nn
import torch.optim as optim
from torchsummary import summary 



import pickle 
import shap
import lime
from lime import lime_tabular
import random
import dalex as dx

import random
from sklearn.preprocessing import StandardScaler
from sklearn.tree import export_graphviz

from imblearn.under_sampling import RandomUnderSampler
from imblearn.over_sampling import SMOTE

import pydotplus
import seaborn as sns

from tabulate import tabulate 


In [2]:
dataSetIndex = 4
needToMakePictureOfTrees = 0
''' ---------------------------------------------------------- '''
dataSetFilePath = ""
dataSetName = ""
dataSetResultDirectory = "./"

if(dataSetIndex == 0):
    dataSetFilePath = "./heartDisease/0_statLog_dataSet.csv"
    dataSetName = "SateLog_DataSet"
elif (dataSetIndex == 1):
    dataSetFilePath = "./heartDisease/1_heart_statlog_cleveland_hungary_final.csv"
    dataSetName = "ALL_StateLog_CleveLand_Hungary"
elif (dataSetIndex == 2):
    dataSetFilePath = "./heartDisease/2_cleveland.csv"
    dataSetName = "Cleveland"
elif (dataSetIndex == 3):
    dataSetFilePath = "./heartDisease/3_framingham.csv"
    dataSetName = "framingham"
elif (dataSetIndex == 4):
    dataSetFilePath = "./heartDisease/4_CardiacPrediction.xlsx"
    dataSetName = "CardiacPrediction"
elif (dataSetIndex == 5):
    dataSetFilePath = "./heartDisease/5_CardiacPredictionLessDimensions.xlsx"
    dataSetName = "CardiacPrediction"
elif (dataSetIndex == 6):
    dataSetFilePath = "./heartDisease/6_CardiacPredictionFewDimensions.xlsx"
    dataSetName = "CardiacPrediction"
else:
    dataSetFilePath = ""
    dataSetName = ""

if(dataSetIndex==4 or dataSetIndex==5 or dataSetIndex==6):
    #fileData = pd.read_excel(dataSetFilePath, sheet_name='CoroHeartDis')
    fileData = pd.read_excel(dataSetFilePath)
else:
    fileData = pd.read_csv(dataSetFilePath)

print("Shape of fileData: {}".format(fileData.shape))
print("Column Headings: {}".format(fileData.__dataframe__().column_names()))
print("Number of Records: {}".format(fileData.__dataframe__().num_rows()))


missingValues = fileData.isnull().any().sum()
print(f"\nNumber of Missing Values: {missingValues}")

num_rows_before = fileData.shape[0] 
fileData.drop_duplicates(inplace=True) 
num_rows_after = fileData.shape[0] 
num_duplicates_removed = num_rows_before - num_rows_after
print(f"Number of duplicate records removed: {num_duplicates_removed}")
fileData = fileData.dropna()
print("Shape of fileData: {}".format(fileData.shape))   
fileData = fileData.fillna(0) 

print("Shape of fileData End: {}".format(fileData.shape))



finalResultTable = [ ['Index', 'Method', 'Accuracy %','Recall %','Precision %','F1 Score','AUC'], ]  

X = fileData.drop(fileData.__dataframe__().column_names()[-1], axis=1)  # Features
Y = fileData[fileData.__dataframe__().column_names()[-1]]  # Labels

columns = fileData.__dataframe__().column_names() 
totalRecords = (fileData.__dataframe__().num_rows())

print("\n")
print("columns of x:: {} \n\n and features of X: {}".format(len(X.columns), X.columns))

dataSetResultDirectory = "./"
dataSetResultDirectory += ("DatasetResults_PyTorch_" + dataSetName)
dataSetResultDirectory += "/"
if not os.path.isdir(dataSetResultDirectory):
    os.makedirs(dataSetResultDirectory)

dataSetName += " {}".format(fileData.shape)



Shape of fileData: (37079, 51)
Column Headings: Index(['SEQN', 'Gender', 'Age', 'Annual-Family-Income',
       'Ratio-Family-Income-Poverty', 'X60-sec-pulse', 'Systolic', 'Diastolic',
       'Weight', 'Height', 'Body-Mass-Index', 'White-Blood-Cells',
       'Lymphocyte', 'Monocyte', 'Eosinophils', 'Basophils', 'Red-Blood-Cells',
       'Hemoglobin', 'Mean-Cell-Vol', 'Mean-Cell-Hgb-Conc.',
       'Mean-cell-Hemoglobin', 'Platelet-count', 'Mean-Platelet-Vol',
       'Segmented-Neutrophils', 'Hematocrit', 'Red-Cell-Distribution-Width',
       'Albumin', 'ALP', 'AST', 'ALT', 'Cholesterol', 'Creatinine', 'Glucose',
       'GGT', 'Iron', 'LDH', 'Phosphorus', 'Bilirubin', 'Protein', 'Uric.Acid',
       'Triglycerides', 'Total-Cholesterol', 'HDL', 'Glycohemoglobin',
       'Vigorous-work', 'Moderate-work', 'Health-Insurance', 'Diabetes',
       'Blood-Rel-Diabetes', 'Blood-Rel-Stroke', 'CoronaryHeartDisease'],
      dtype='object')
Number of Records: 37079

Number of Missing Values: 0
Number o

In [3]:
print("Shape of fileData: {} , target Len:{}".format(fileData.shape, len(Y)))
print("X: {} , Y:{}".format(X.shape, Y.shape))
#print("\n\nX: head:: \n{}".format(X.head()))
#print("\n\nY: head::\n {}".format(Y.head()))


# Split dataset into training set and test set
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.3, stratify=Y, random_state=42)


positiveClass =  0
negativeClass = 0
for i in y_train:
    if(i == 0):
        negativeClass += 1
    if(i == 1):
        positiveClass += 1
print("Before SMOTE Train DataSet Positive Class Records:: {}".format(positiveClass)) 
print("Before SMOTE Train DataSet Negative Class Records:: {}".format(negativeClass)) 
print("Before SMOTE Train DataSet Total Records:: {}".format(positiveClass + negativeClass)) 


print("Target Column Name:: {} \n".format(fileData.__dataframe__().column_names()[-1]))


rus = RandomUnderSampler(random_state=42)
smote = SMOTE(random_state=42)

'''
# Undersample the majority class
X_train, y_train = rus.fit_resample(X_train, y_train)
X_test, y_test = rus.fit_resample(X_test, y_test)
'''


print("\n \n\n\n")

# Oversample the minority class using SMOTE
X_train, y_train = smote.fit_resample(X_train, y_train)
#X_test, y_test = rus.fit_resample(X_test, y_test)
X_test, y_test = smote.fit_resample(X_test, y_test) 


scaler = StandardScaler().fit(X_train)
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)
#Y_train_scaled = scaler.transform(y_train)
#Y_test_scaled = scaler.transform(y_test)

 


print("\n X Train: Shape:: {}".format(X_train.shape))
print(" X Test: Shape:: {}".format(X_test.shape))  
 
 
positiveClass =  0
negativeClass = 0
for i in y_train:
    if(i == 0):
        negativeClass += 1
    if(i == 1):
        positiveClass += 1
print("After SMOTE Train DataSet Positive Class Records:: {}".format(positiveClass)) 
print("After SMOTE Train DataSet Negative Class Records:: {}".format(negativeClass)) 
print("After SMOTE Train DataSet Total Records:: {}".format(positiveClass + negativeClass)) 

print("\n\n") 


positiveClass =  0
negativeClass = 0
for i in y_test:
    if(i == 0):
        negativeClass += 1
    if(i == 1):
        positiveClass += 1
print("After SMOTE Test DataSet Positive Class Records:: {}".format(positiveClass)) 
print("After SMOTE Test DataSet Negative Class Records:: {}".format(negativeClass)) 
print("After SMOTE Test DataSet Total Records:: {}".format(positiveClass + negativeClass)) 



Shape of fileData: (37079, 51) , target Len:37079
X: (37079, 50) , Y:(37079,)
Before SMOTE Train DataSet Positive Class Records:: 1056
Before SMOTE Train DataSet Negative Class Records:: 24899
Before SMOTE Train DataSet Total Records:: 25955
Target Column Name:: CoronaryHeartDisease 


 





Could not find the number of physical cores for the following reason:
found 0 physical cores < 1
Returning the number of logical cores instead. You can silence this warning by setting LOKY_MAX_CPU_COUNT to the number of cores you want to use.
  File "c:\Users\Mubashir Iqbal\AppData\Local\Programs\Python\Python310\lib\site-packages\joblib\externals\loky\backend\context.py", line 282, in _count_physical_cores
    raise ValueError(f"found {cpu_count_physical} physical cores < 1")



 X Train: Shape:: (49798, 50)
 X Test: Shape:: (21344, 50)
After SMOTE Train DataSet Positive Class Records:: 24899
After SMOTE Train DataSet Negative Class Records:: 24899
After SMOTE Train DataSet Total Records:: 49798



After SMOTE Test DataSet Positive Class Records:: 10672
After SMOTE Test DataSet Negative Class Records:: 10672
After SMOTE Test DataSet Total Records:: 21344


In [4]:
def makeCorrelationPic(correlationMatrix, numberOfTopFeatures, targetColumnName):     
    correlation_values = correlationMatrix.abs()
    sorted_correlation = correlation_values.unstack().sort_values(ascending=False)
    sorted_correlation = sorted_correlation[sorted_correlation != 1.0]

    num_features = numberOfTopFeatures  # Number of top features to display
    top_features = sorted_correlation.head(num_features)
    print("Top", num_features, "features based on correlation:")
    print(top_features)
 
    top_features = correlationMatrix.abs().nlargest(numberOfTopFeatures, targetColumnName)[targetColumnName].index
    top_correlation_matrix = correlationMatrix.loc[top_features, top_features]

    mplot.figure(figsize=(10, 8))
    sns.heatmap(top_correlation_matrix, annot=True, cmap='coolwarm')
    # Set the title of the plot
    mplot.title('Correlation Heatmap ({})'.format(dataSetName))
    
    picturePath = "Correlation_Matrix_DateSetName_{}.png".format(dataSetName)
    mplot.savefig(picturePath,  dpi=300, bbox_inches='tight')
    mplot.show()
    mplot.close()

def makeConfusionMatrixPic(method, dataSet, classifierObj , X_test, y_test, predicted_Y):
    display = ConfusionMatrixDisplay.from_estimator(classifierObj, X_test, y_test, display_labels=['Healthy', "Heart Disease"], cmap=mplot.cm.Blues) #, normalize="true"
    display.ax_.set_title("Confusion Matrix ({} Model)".format(method))
    display.ax_.set_xlabel('\nPredicted Values')
    display.ax_.set_ylabel('Actual Values ')


    accuracyString ="Accuracy {}: {:.2f}".format(method, accuracy_score(y_test, predicted_Y)*100.0 ) 
    recallString =  'Recall {}: {:.2f}'.format(method, recall_score(y_test, predicted_Y) * 100.0)
    precisionString = 'Precision {}: {:.2f}'.format(method, precision_score(y_test, predicted_Y) * 100.0) 
    dataSetString = "Dataset: {}".format(dataSet)

    
    if(classifierObj.n_features_in_ > 10):
        featureListString = 'Total Features: {}'.format(classifierObj.n_features_in_) 
    else:
        featureListString = 'Features: {}'.format(classifierObj.feature_names_in_) 
    
    display.figure_.text(0.010, -0.05,  accuracyString, horizontalalignment='left', wrap=False )  
    display.figure_.text(0.010, -0.09,  recallString, horizontalalignment='left', wrap=False )      
    display.figure_.text(0.010, -0.13,  precisionString, horizontalalignment='left', wrap=False ) 
    display.figure_.text(0.010, -0.17,  dataSetString, horizontalalignment='left', wrap=False ) 
    display.figure_.text(0.010, -0.28,  featureListString, horizontalalignment='left', wrap=False ) 
 
    picturePath = "{}Confusion_Matrix_{}_{}.png".format(dataSetResultDirectory, method, dataSetName)
    mplot.savefig(picturePath,  dpi=300, bbox_inches='tight')
    #print("{} Confusion Matrix saved:: path: {}".format(method, picturePath))
    #os.startfile(picturePath)
    mplot.show()
    mplot.close()


In [5]:
# Convert to PyTorch tensors
import torch
import torch.nn as nn
import torch.optim as optim
from torchsummary import summary
import numpy as np
import pandas as pd
 
X_train_tensor = torch.tensor(X_train.to_numpy(), dtype=torch.float32).cuda()
y_train_tensor = torch.tensor(y_train.to_numpy(), dtype=torch.float32).cuda()
X_test_tensor = torch.tensor(X_test.to_numpy(), dtype=torch.float32).cuda()
y_test_tensor = torch.tensor(y_test.to_numpy(), dtype=torch.float32).cuda()
#print(X_train_tensor)
#print(y_train_tensor) 
# Define a custom dataset
class CustomDataset(Dataset):
    def __init__(self, features, labels):
        self.features = features
        self.labels = labels

    def __len__(self):
        return len(self.features)

    def __getitem__(self, idx):
        return self.features[idx], self.labels[idx]
    
train_dataset = CustomDataset(X_train_tensor, y_train_tensor)
test_dataset = CustomDataset(X_test_tensor, y_test_tensor)
print(train_dataset)
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)
print(train_loader)


In [6]:
features = X_train_tensor 
target = y_train_tensor
 
  
print("X_train shape: {}   and dType: {}".format(X_train.shape, len(X_train.columns)))
print("y_train shape: {}   and dType: {}".format(y_train.shape, y_train.dtype))  

print("X_train_tensor shape: {}   and dType: {}".format(X_train_tensor.shape, X_train_tensor.dtype))
print("y_train_tensor shape: {}   and dType: {}".format(y_train_tensor.shape, y_train_tensor.dtype)) 

print("X_test_tensor shape: {}   and dType: {}".format(X_test_tensor.shape, X_test_tensor.dtype))
print("y_test_tensor shape: {}   and dType: {}".format(y_test_tensor.shape, y_test_tensor.dtype)) 

print("features shape: {}   and dType: {}".format(features.shape, features.dtype))
print("target shape: {}   and dType: {}".format(target.shape, target.dtype)) 

X_train shape: (49798, 50)   and dType: 50
y_train shape: (49798,)   and dType: int64
X_train_tensor shape: torch.Size([49798, 50])   and dType: torch.float32
y_train_tensor shape: torch.Size([49798])   and dType: torch.float32
X_test_tensor shape: torch.Size([21344, 50])   and dType: torch.float32
y_test_tensor shape: torch.Size([21344])   and dType: torch.float32
features shape: torch.Size([49798, 50])   and dType: torch.float32
target shape: torch.Size([49798])   and dType: torch.float32


In [7]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchsummary import summary
import numpy as np
import pandas as pd
 

# Define the attention layer
class AttentionLayer(nn.Module):
    def __init__(self, input_size):
        super(AttentionLayer, self).__init__()
        self.attention = nn.MultiheadAttention(embed_dim=input_size, num_heads=1)
    def forward_2(self, q, v):
        attention_weights, _ = self.attention(q.unsqueeze(0), v.unsqueeze(0), v.unsqueeze(0))
        return attention_weights.squeeze(0)
    
    def forward(self, q, v):
        attention = self.attention(q, v, v)
        return attention

# Define the model
class CustomModel(nn.Module):
    def __init__(self, input_size): 
        super(CustomModel, self).__init__()
         # Traditional neural network part
        self.fc1 = nn.Linear(input_size, input_size)
        self.relu1 = nn.ReLU()
        self.fc2 = nn.Linear(input_size, input_size)
        self.relu2 = nn.ReLU()
        self.fc3 = nn.Linear(input_size, input_size)
        self.relu3 = nn.ReLU()
        self.batch_norm = nn.BatchNorm1d(input_size)
        # Attention layer
        self.attention = AttentionLayer(input_size)
        # Output layer
        self.fc4 = nn.Linear(input_size*2, input_size)
        self.relu4 = nn.ReLU()
        self.batch_norm2 = nn.BatchNorm1d(input_size)
        self.fc5 = nn.Linear(input_size, input_size//2)
        self.relu5 = nn.ReLU()
        self.fc6 = nn.Linear(input_size//2, 1)
        self.sigmoid = nn.Sigmoid()

    def forward(self, x): 
     # Traditional neural network part
        x = self.fc1(x)
        x = self.relu1(x)
        x = self.fc2(x)
        x = self.relu2(x)
        x = self.fc3(x)
        x = self.relu3(x)
        x = self.batch_norm(x)
        # Detach the gradient for interpretability
        x_detached = x.detach()
        # Attention layer
        attention = self.attention(x_detached, x_detached) 
        # Combine the output of the attention layer with the previous layer
        x = torch.cat([x, attention], dim=1)
        # Output layer
        x = self.fc4(x)
        x = self.relu4(x)
        x = self.batch_norm2(x)
        x = self.fc5(x)
        x = self.relu5(x)
        x = self.fc6(x)
        output = self.sigmoid(x)

        return output
     






    def predict_proba(self, x): 
        logits = self.forward(x)
        probs = torch.sigmoid(logits) # .detach().cpu().numpy()
        return probs
    

# Instantiate the model
input_size = X_train_tensor.shape[1]  # Assuming 'features' is a torch tensor
print(input_size)
model = CustomModel(input_size).cuda()
optimizer = optim.AdamW(model.parameters(), lr=0.001)
criterion = nn.BCELoss()
 


50


In [8]:
from torchsummary import summary

print(torch.cuda.memory_summary(device=None, abbreviated=False))


|===========================================================================|
|                  PyTorch CUDA memory summary, device ID 0                 |
|---------------------------------------------------------------------------|
|            CUDA OOMs: 0            |        cudaMalloc retries: 0         |
|===========================================================================|
|        Metric         | Cur Usage  | Peak Usage | Tot Alloc  | Tot Freed  |
|---------------------------------------------------------------------------|
| Allocated memory      |  14278 KiB |  14278 KiB |  14278 KiB |      0 B   |
|       from large pool |  13895 KiB |  13895 KiB |  13895 KiB |      0 B   |
|       from small pool |    383 KiB |    383 KiB |    383 KiB |      0 B   |
|---------------------------------------------------------------------------|
| Active memory         |  14278 KiB |  14278 KiB |  14278 KiB |      0 B   |
|       from large pool |  13895 KiB |  13895 KiB |  13895 KiB |

In [9]:
numberOfEpochs = 30
batchSizeOfTraining = 2 
accumulation_steps = 4
import gc

y_train_tensor = y_train_tensor.view(-1, 1) 
torch.cuda.empty_cache()

for epoch in range(numberOfEpochs): 
    output = model(X_train_tensor)
    loss = criterion(output, y_train_tensor) 
    loss.backward()    
    # Accumulate gradients for a specified number of steps
    if (i + 1) % accumulation_steps  == 0:
        optimizer.step()
        optimizer.zero_grad()
        del output, loss
        gc.collect()
        torch.cuda.empty_cache()
         
 

OutOfMemoryError: CUDA out of memory. Tried to allocate 9.24 GiB. GPU 0 has a total capacty of 4.00 GiB of which 0 bytes is free. Of the allocated memory 9.33 GiB is allocated by PyTorch, and 23.94 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF

In [ ]:
model.eval()
with torch.no_grad():
    y_pred = model(X_test_tensor).cuda() #.numpy()

# Convert the list to a PyTorch tensor
binary_predictions_tensor = torch.tensor([1 if pred > 0.5 else 0 for pred in y_pred])
binary_predictions_tensor = binary_predictions_tensor.cuda()

from sklearn.metrics import accuracy_score

y_test_cpu = y_test_tensor.cpu().view(-1, 1).numpy()
binary_predictions_cpu = binary_predictions_tensor.cpu().numpy()

loss = criterion(torch.tensor(y_pred), y_test_tensor.view(-1, 1).cuda()).item()
accuracy = accuracy_score(y_test_cpu, binary_predictions_cpu)
recall_value = recall_score(y_test_cpu, binary_predictions_cpu)
precision_value = precision_score(y_test_cpu, binary_predictions_cpu)
f1_score_value = f1_score(y_test_cpu, binary_predictions_cpu)
auc_value = roc_auc_score(y_test_cpu, binary_predictions_cpu)

# Print the results
print('Test loss: {}'.format(loss*100))
print('Test accuracy: {}'.format(accuracy*100))
print('Test recall: {}'.format(recall_value*100))
print('Test precision: {}'.format(precision_value*100))
print('Test F1 score: {}'.format(f1_score_value*100))
print('Test AUC: {}'.format(auc_value*100))

In [ ]:
display = ConfusionMatrixDisplay.from_predictions(y_test_cpu, binary_predictions_cpu, display_labels=['Healthy', 'Heart Disease'], cmap=mplot.cm.Blues)

#display = ConfusionMatrixDisplay.from_estimator(model, X_test_scaler, y_test, display_labels=['Healthy', "Heart Disease"], cmap=mplot.cm.Blues) #, normalize="true"
method = "MLP with Attention layers"
display.ax_.set_title("Results {} Model".format(method))
display.ax_.set_xlabel('\nPredicted Values')
display.ax_.set_ylabel('Actual Values ')

accuracyString ="Accuracy {}: {:.2f}".format(method, (accuracy*100.0) ) 
recallString =  'Recall {}: {:.2f}'.format(method, (recall_value*100.0))
precisionString = 'Precision {}: {:.2f}'.format(method, (precision_value*100.0)) 
dataSetString = "F1 Score: {}".format((f1_score_value*100.0))
featureListString = "AUC Score: {}".format(auc_value*100.0)
numberOfEpochsString = "Number of Epoches: {}".format(numberOfEpochs)
batchSizeOfTrainingString = "BatchSize for Epoch: {}".format(batchSizeOfTraining)

display.figure_.text(0.010, -0.05,  accuracyString, horizontalalignment='left', wrap=False )  
display.figure_.text(0.010, -0.09,  recallString, horizontalalignment='left', wrap=False )      
display.figure_.text(0.010, -0.13,  precisionString, horizontalalignment='left', wrap=False ) 
display.figure_.text(0.010, -0.17,  dataSetString, horizontalalignment='left', wrap=False ) 
display.figure_.text(0.010, -0.21,  featureListString, horizontalalignment='left', wrap=False ) 
display.figure_.text(0.010, -0.28,  numberOfEpochsString, horizontalalignment='left', wrap=False ) 
display.figure_.text(0.010, -0.32,  batchSizeOfTrainingString, horizontalalignment='left', wrap=False ) 
 
picturePath = "{}Model_Evaluation_{}_{}_Epoch_{}.png".format(dataSetResultDirectory, method, dataSetName, numberOfEpochs)
mplot.savefig(picturePath,  dpi=300, bbox_inches='tight') 

mplot.show()
mplot.close()





In [ ]:
percentage = 0.10
numberOfFeatures = int(len(features) *percentage) 
featuresForShap = X_train.columns  
print("Total Features: {} -> Selected for SHAP:: {}".format(len(features), numberOfFeatures))
numberOftest = int(len(X_test_tensor) * percentage) 
testForShap = X_test_tensor[0:numberOftest] 
print(" testForShap length: {}".format( len(testForShap)))
 
# Create SHAP explainer
shap_explainer = shap.DeepExplainer(model, testForShap)
shap_values = shap_explainer.shap_values(testForShap)
shap.summary_plot(shap_values, features=testForShap, feature_names=featuresForShap, show=False)

ax = mplot.gca() 
#ax.set_title("XAI SHAP Explainer ({} Model)".format(method) ,fontsize=16, fontweight='bold')     
ax.set_title("XAI SHAP Explainer " ,fontsize=16, fontweight='bold')     

dataSetString = "Dataset:  {}".format(dataSetName)
testingDatasetString ="length of Testing Set: {}".format(len(testForShap))
shapTypeString ="SHAP Type: {}".format(repr(shap_explainer)) 
ax.figure.text(0.020, -0.05,  dataSetString, horizontalalignment='left', wrap=False )  
ax.figure.text(0.020, -0.09,  testingDatasetString, horizontalalignment='left', wrap=False )   
ax.figure.text(0.020, -0.13,  shapTypeString, horizontalalignment='left', wrap=False )   

picturePath = "{}XAI_SHAP_Explainer2_{}_{}_numberOfSamples_{}.png".format(dataSetResultDirectory, method, dataSetName, len(testForShap))
mplot.savefig(picturePath,  dpi=300, bbox_inches='tight')
mplot.show()
mplot.close() 






In [ ]:
import dalex as dx 
#torch.cuda.empty_cache()

X_data = X_train_tensor[0:10]
Y_data = y_train_tensor[0:10]

 

# Convert PyTorch tensors back to pandas DataFrame
X_train_df = pd.DataFrame(X_data.cpu().numpy(), columns=X_train.columns)
y_train_df = pd.DataFrame(Y_data.cpu().numpy(), columns=[y_train.name])

# Convert PyTorch model to a Dalex Explainer
explainer = dx.Explainer(model, X_train_df, y_train_df, label="CustomModel", verbose=True)

# Create a DALEX model wrapper for the PyTorch model
model_dalex = dx.ModelWrapper(model, predict_function=model.predict_proba)

# Plot the variable importance
variable_importance_plot = explainer.model_parts()
variable_importance_plot.plot()
 
# Convert PyTorch model to a Dalex Explainer
explainer = dx.Explainer(model, X_data, Y_data, label="CustomModel", verbose=True, predict_function=model.predict_proba)

# Create a DALEX model wrapper for the PyTorch model
#model_dalex = dx.ModelWrapper(model, predict_function=model.predict_proba)

# Plot the variable importance
variable_importance_plot = explainer.model_parts()
variable_importance_plot.plot()

 


In [ ]:
import lime
from lime import lime_tabular
 
X_data = X_train_tensor[0:10]
Y_data = y_train_tensor[0:10] 
X_train_df = pd.DataFrame(X_data.cpu().detach().numpy(), columns=X_train.columns)
y_train_df = pd.DataFrame(Y_data.cpu().detach().numpy(), columns=[y_train.name])

# Create a function that uses the PyTorch model for predictions
def pytorch_predict_fn(data):
    data_tensor = torch.tensor(data, dtype=torch.float32).cuda()
    return model.predict_proba(data_tensor).cpu().detach().numpy()




  
# Create a LimeTabularExplainer
explainer = lime_tabular.LimeTabularExplainer(X_train_df.values,  feature_names=X_train.columns.tolist(),
                                              class_names=['CoronaryHeartDisease'],  # Adjust class names accordingly
                                              discretize_continuous=True, mode='classification')

# Choose an instance for which you want an explanation
instance_idx = 0
instance = X_test_tensor[instance_idx].cpu().detach().numpy()
instance = instance.flatten()  # or instance.squeeze()
print(instance)
print(len(instance))
# Get an explanation for the instance
explanation = explainer.explain_instance(instance, pytorch_predict_fn, num_features=len(X_train.columns))

# Plot the explanation
explanation.show_in_notebook(show_table=True)


In [ ]:
import matplotlib.pyplot as plt
import pdpbox  
from pdpbox import pdp 

from pdpbox import info_plots
#fig, axes, summary_df = info_plots.TargetPlot( df=fileData, feature=fileData[['Gender']], feature_name='Gender', target=Y)
#fig, axes, summary_df = info_plots.TargetPlot( df=fileData, feature=fileData['Gender'], feature_name='Gender', target=Y)
X_data = X_train[0:10]

feature = 'sex'
feature_name = 'Gender' 

 
# Example for generating PDP for a specific feature (replace 'feature_name' with the actual feature name)
#pdp_feature = pdp.pdp_isolate(model=model, dataset=X_data, model_features=X_data.columns, feature_names=['feature_name'])
pdp_feature = pdp.pdp_plot (model=model, dataset=X_data)

# Plot the PDP
pdpbox.pdp.pdp_plot(pdp_feature, feature_name)







plt.show()
 
  




In [ ]:
from pdpbox import info_plots
import matplotlib.pyplot as plt

# Assuming 'Gender' is a column in your DataFrame
feature_name = 'Gender'

fig, axes, summary_df = info_plots.TargetPlot(df=fileData, feature=fileData['Gender'], feature_name='Gender', target=Y)

plt.show()


In [ ]:
import dalex as dx
dx.__version__
numberOftest = int((len(y_test_tensor) * percentage))
y_trainForXAI = y_test_tensor[0:numberOftest]  
#print(numberOftest)
#print(y_trainForXAI)

def predict_function(model, data): 
    #print("\n\n data :: {}".format(data)) 
    data_tensor = torch.from_numpy(data)     
    #print("\n\n data_tensor :: {}".format(data_tensor)) 
    predValue = model.predict_proba(data_tensor)
    print("\n\npredict_function :: {}".format(predValue)) 
    predValue = predValue.detach().cpu().numpy()
    print("\n\npredict_function :: {}".format(predValue[:, 1]))   
    return predValue[:, 1]
  

# Convert testForShap and y_trainForXAI to NumPy arrays
test_for_shap_numpy = testForShap.detach().cpu().numpy()
y_train_for_xai_numpy = y_trainForXAI.detach().cpu().numpy()
 

explainer = dx.Explainer(model, test_for_shap_numpy, y_train_for_xai_numpy,  predict_function=predict_function, label='CoronaryHeartDisease', model_type='regression')


print("Working ")


explainer.model_performance()


In [ ]:

explainer.model_parts().plot()

In [ ]:
from shapash.explainer.consistency import Consistency
cns = Consistency()

cns.compile(model , x=X_test_tensor  )

In [ ]:

# Convert your training data to NumPy
X_train_numpy = testForShap.cuda().numpy()

# Define a predict function for your PyTorch model
def predict_function(data):
    input_tensor = torch.tensor(data, dtype=torch.float32)
    output = model(input_tensor)
    return output.detach().numpy()

# Create a LIME explainer
explainer = lime_tabular.LimeTabularExplainer(X_train_numpy, feature_names=featuresForShap, mode='regression')
# Get LIME explanations for a single instance
explanation = explainer.explain_instance(X_train_numpy[0], predict_function, num_features=len(featuresForShap))
# Visualize the LIME explanation
lime_summary_plot = explanation.as_pyplot_figure()

# Save Lime explanation summary plot as a PNG file
lime_picture_path = "{}XAI_LIME_Explainer_{}_{}_numberOfSamples_{}.png".format(dataSetResultDirectory, method, dataSetName, len(X_train_numpy))
lime_summary_plot.savefig(lime_picture_path, dpi=300, bbox_inches='tight')
plt.show()
plt.close()  # Close the figure to release resources and avoid displaying the plot
 



In [ ]:
import lime
from lime import lime_tabular
 
# Define a function that takes numpy arrays as input and returns model predictions
def predict_function(data):
     return np.zeros(data.shape[0]) 


# Convert your training data to NumPy
X_train_numpy = X_train_tensor.cpu().detach().numpy()

# Create a LIME explainer
explainer = lime_tabular.LimeTabularExplainer(X_train_numpy,  feature_names=featuresForShap,  mode='regression')

# Get LIME explanations for a single instance
explanation = explainer.explain_instance(testForShap[0], predict_function,  num_features=len(featuresForShap))

# Visualize the LIME explanation
lime_summary_plot = explanation.as_pyplot_figure()

lime_summary_plot.set_title("XAI SHAP Explainer ({} Model)".format(method) ,fontsize=16, fontweight='bold')     

dataSetString = "Dataset:  {}".format(dataSetName)
testingDatasetString ="length of Testing Set: {}".format(len(testForShap))
shapTypeString ="SHAP Type: {}".format(repr(shap_explainer)) 
lime_summary_plot.figure.text(0.020, -0.05,  dataSetString, horizontalalignment='left', wrap=False )  
lime_summary_plot.figure.text(0.020, -0.09,  testingDatasetString, horizontalalignment='left', wrap=False )   
lime_summary_plot.figure.text(0.020, -0.13,  shapTypeString, horizontalalignment='left', wrap=False )   

# Save Lime explanation summary plot as a PNG file
lime_picture_path = "{}XAI_LIME_Explainer_{}_{}_numberOfSamples_{}.png".format(dataSetResultDirectory, method, dataSetName, len(testForShap))
lime_summary_plot.savefig(lime_picture_path, dpi=300, bbox_inches='tight')
plt.show()
plt.close()  # Close the figure to release resources and avoid displaying the plot
 

In [ ]:
percentage = 0.07
numberOfFeatures = int(len(features) *percentage)
print("Total Features: {} -> Selected for SHAP:: {}".format(len(features), numberOfFeatures))
featuresForShap = X_train.columns #features[0:numberOfFeatures]
#print(" Features Name: {}".format(  featuresForShap))

numberOftest = int(len(X_test_tensor) * percentage)
print("Total Test: {} -> Selected for SHAP:: {}".format(len(X_test_tensor), numberOftest))
#testForShap = X_test_tensor[0:len(featuresForShap)]
testForShap = X_test_tensor[0:numberOftest]
#print(" testForShap Name: {}".format(  testForShap))
testForShap_numpy = testForShap.cpu().detach().numpy()

# Convert the NumPy array back to a PyTorch tensor for SHAP values computation
testForShap_tensor2 = torch.from_numpy(testForShap_numpy)


 
def makeSHAPreport(method, model):
    shap_explainer = shap.Explainer(model, feature_names=featuresForShap, masker=shap.maskers.Independent(data=testForShap_numpy)) 
    shap_values = shap_explainer.shap_values(testForShap)  
    shap.summary_plot(shap_values, testForShap_numpy, featuresForShap, show=False)
    ax = mplot.gca() 
    ax.set_title("XAI SHAP Explainer ({} Model)".format(method) ,fontsize=16, fontweight='bold')     

    dataSetString = "Dataset:  {}".format(dataSetName)
    testingDatasetString ="length of Testing Set: {}".format(len(testForShap_numpy))
    shapTypeString ="SHAP Type: {}".format(repr(shap_explainer)) 
    ax.figure.text(0.020, -0.05,  dataSetString, horizontalalignment='left', wrap=False )  
    ax.figure.text(0.020, -0.09,  testingDatasetString, horizontalalignment='left', wrap=False )   
    ax.figure.text(0.020, -0.13,  shapTypeString, horizontalalignment='left', wrap=False )   

    picturePath = "{}XAI_SHAP_Explainer_{}_{}_numberOfSamples_{}.png".format(dataSetResultDirectory, method, dataSetName, len(testForShap_numpy))
    mplot.savefig(picturePath,  dpi=300, bbox_inches='tight')
    #mplot.savefig(picturePath,  dpi=300) 
    mplot.show()
    #os.startfile(picturePath)
    mplot.close()

In [ ]:
from sklearn.metrics import confusion_matrix, classification_report
 
y_test_cpu = y_test_tensor.cpu().numpy()

# Convert to binary predictions (assuming binary classification)
binary_predictions = [1 if pred > 0.5 else 0 for pred in y_pred]

# Draw confusion matrix
cm = confusion_matrix(y_test_cpu, binary_predictions)
mplot.figure(figsize=(8, 6))
sns.heatmap(cm, annot=True, fmt="d", cmap="Blues", cbar=False)
mplot.xlabel("Predicted")
mplot.ylabel("True")
picturePath = "{}Confusion_Matrix_{}_{}.png".format(dataSetResultDirectory, "MLP+Attention", dataSetName)
mplot.savefig(picturePath,  dpi=300, bbox_inches='tight')
mplot.show()
mplot.close()

# Print classification report
print(classification_report(y_test_cpu, binary_predictions))


In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
import pandas as pd

import os
os.environ['CUDA_LAUNCH_BLOCKING'] = '1'
os.environ['CUBLAS_WORKSPACE_CONFIG'] = ':16:8'
  



  

In [ ]:

# Evaluation
model.eval()
predictions = []
true_labels = []

with torch.no_grad():
    for inputs, labels in test_loader:
        inputs, labels = inputs.to(device), labels.to(device)
        outputs = model(inputs)
        predictions.extend(outputs.cpu().numpy())
        true_labels.extend(labels.cpu().numpy())

# Convert to binary predictions (assuming binary classification)
binary_predictions = [1 if pred > 0.5 else 0 for pred in predictions]

# Draw confusion matrix
cm = confusion_matrix(true_labels, binary_predictions)
mplot.figure(figsize=(8, 6))
sns.heatmap(cm, annot=True, fmt="d", cmap="Blues", cbar=False)
mplot.xlabel("Predicted")
mplot.ylabel("True")
mplot.show()

# Print classification report
print(classification_report(true_labels, binary_predictions))